In [8]:
# Script that returns statistics on File Count and File Size for Experimental Strategies, broken down by Project
# LEGACY

In [9]:
import requests
import json

def GetProjects(): # Get List of Projets and assign in variable called p_list
    project_list = 'https://gdc-api.nci.nih.gov/legacy/projects?size=1000'

    projects = requests.get(project_list)
    data = json.loads(projects.text)
    current_data = data['data']['hits']

    p_list = []
    
    for i in range(len(current_data)):
        p_list.append(current_data[i]['project_id'])
    return p_list

In [10]:
def GetRequests(PList,ES): # Create API Requests and append them to a list titled 'Requests'

    API_URL = 'https://gdc-api.nci.nih.gov/legacy/'
    Endpoint = 'files'
    Fields = 'file_size'
    Size = '100000'
    
    Requests = []
    for Project in PList:
        Filters = '%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.project.project_id%22%2C%22value%22%3A%5B%22'+Project+'%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.experimental_strategy%22%2C%22value%22%3A%5B%22'+ES+'%22%5D%7D%7D%5D%7D'
        Request = API_URL + Endpoint + '?' + 'filters=' + Filters + '&' + 'fields=' + Fields + '&' + 'size=' + Size
        Requests.append(Request)
    return Requests

In [11]:
def ExtractData(Request_List): # Excute the API Requests and append to a list titled 'Data'
    Data = []
    for RQ in Request_List:
            r = requests.get(RQ)
            data = json.loads(r.text)
            current_data = data['data']['hits']
            Data.append(current_data)
    return Data

In [12]:
def GetFile_Size_Count(Data): # Extract data to get file count and file size for each set of projects
    File_Count = []
    File_Size = []
    for P in range(len(Data)):
        File_Count.append(len(Data[P]))

        File_Size_List = []
        for results in range(len(Data[P])):
            File_Size_List.append(Data[P][results]['file_size'])

        File_Size_List_Int = map(int, File_Size_List)
        File_Size.append(sum(File_Size_List_Int))
        
    return File_Count,File_Size

In [14]:
import pandas as pd

# Get Projects
Project_List = GetProjects()

# List Experimental Strategies Available
ES_List = ['Genotyping%20array','RNA-Seq','DNA-Seq','miRNA-Seq','Methylation%20array','Protein%20expression%20array',\
'WXS','CGH%20array','VALIDATION','Gene%20expression%20array','WGS','MSI-Mono-Dinucleotide%20Assay',\
'miRNA%20expression%20array','Mixed%20strategies','AMPLICON','Exon%20array','Total%20RNA-Seq',\
'Capillary%20sequencing','Bisulfite-Seq']

# Loop through experimental strategies creating a data frame of file size and counts for each one
ES_Results = []
for ES in ES_List:
    RQs = GetRequests(Project_List, ES)
    Data = ExtractData(RQs)
    Size_Count = GetFile_Size_Count(Data)
    ES = ES.replace('%20', ' ')
    DataFrame = pd.DataFrame(index=Project_List, data={ES + ' File Count': Size_Count[0], ES + ' File Size (Bytes)': Size_Count[1]})
    ES_Results.append(DataFrame)

# Create Data Frame of All Experimental Strategies
ES_Final = pd.merge(ES_Results[0], ES_Results[1], left_index=True, right_index=True)
for i in range(len(ES_List)-2):
    ES_Final = pd.merge(ES_Final, ES_Results[i+2], left_index=True, right_index=True)
ES_Final

,Genotyping array File Count,Genotyping array File Size (Bytes),RNA-Seq File Count,RNA-Seq File Size (Bytes),DNA-Seq File Count,DNA-Seq File Size (Bytes),miRNA-Seq File Count,miRNA-Seq File Size (Bytes),Methylation array File Count,Methylation array File Size (Bytes),...,AMPLICON File Count,AMPLICON File Size (Bytes),Exon array File Count,Exon array File Size (Bytes),Total RNA-Seq File Count,Total RNA-Seq File Size (Bytes),Capillary sequencing File Count,Capillary sequencing File Size (Bytes),Bisulfite-Seq File Count,Bisulfite-Seq File Size (Bytes)
CCLE-OV,0,0,45,664138628179,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-HNSC,10978,266812696692,5410,7502850991199,3314,17406067115,3202,246621628910,2320,34924998499,...,0,0,0,0,0,0,0,0,0,0
TCGA-COAD,9912,244603902941,5024,5570631201682,1290,13549783822,3932,266961248380,2224,22071748939,...,0,0,0,0,156,703787804,0,0,11,416110579241
TCGA-LIHC,7728,189003740097,3470,4868599585403,2591,8307318567,2262,145088736271,1720,25897592491,...,0,0,0,0,0,0,0,0,0,0
CCLE-LCML,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CCLE-UCEC,0,0,4,75940190944,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CCLE-UVM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CCLE-UCS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CCLE-KIRC,0,0,25,351511027149,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-PAAD,3699,90044489484,1464,2100407694250,1383,4932650000,947,66048542412,780,11743608701,...,155,104637480437,0,0,0,0,0,0,0,0


In [16]:
# Save the Data Frame to a file
ES_Final.to_csv(path_or_buf='Legacy_Files_Per_Project.tsv',sep='\t', header=True)